# **Import Libraries**

In [ ]:
!pip install nltk
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob 
from sklearn.feature_extraction.text import TfidfVectorizer
#!pip install xgboost
from xgboost import XGBRegressor 

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# **Reading a dataset**

In [ ]:
data = pd.read_csv(r'../input/jigsaw-toxic-severity-rating/comments_to_score.csv')
data.head()
n = len(data)

In [ ]:
n

In [ ]:
train_data = pd.read_csv(r'../input/jigsaw-toxic-comment-classification-challenge/train.csv')
#train_data = train_data[:n]
train_data.head()
text = data['text']

In [ ]:
score = []
for i in range(len(train_data)):
    x = train_data.iloc[i]
    s = sum(x['toxic':].values)
    score.append(round(s/6, 3))

In [ ]:
train_data['score'] = score
train_data.head()

In [ ]:
#   Sentence length in 
text.str.len().hist()

In [ ]:
# Word length Analysis
import matplotlib.pyplot as plt
arr = [ ]
for i in text:
    
    tmp = i.split(' ')
    arr.append(len(tmp))
plt.grid()
plt.hist(arr)

In [ ]:
nltk.download('stopwords')
stop = set(stopwords.words('english'))
import numpy as np
corpus = []
tmp = []
for i in text : 
    tmp.append(i.split())
    
for i in tmp:
    for words in i :
        corpus.append(words)
from collections import defaultdict
dic = defaultdict(int)
for i in corpus: 
    if i in stop :
        dic[i] = dic[i] + 1
val = dic.values()
val = sorted(val,reverse = True)
val = val[0:11]
d = {}
for i in list(dic.keys()):
    if dic[i] in val: 
        d[i] = dic[i]
# Top 10 Stopwords in corpus
plt.bar(list(d.keys()), list(d.values()))

In [ ]:
#bigrams, trigrams
from nltk.util import ngrams
import collections
tex = ''
#word_tokenize = nltk.download('word_tokenize')
for i in text : 
    tex = tex + str(i.strip('[]'))
tok = tex.split()
b_grams = ngrams(tok, 2) 
t_grams = ngrams(tok,3)
bigrams = collections.Counter(b_grams)
trigrams = collections.Counter(t_grams)
bi= bigrams.most_common(10)
ti= trigrams.most_common(10)

In [ ]:
def plot_ngrams(bi):
    word = []
    idx = []
    for i in range(len(bi)):
        word.append(str(bi[i][0]))
        idx.append(bi[i][1])
    plt.figure(figsize = (10,8))
    plt.bar(word,idx)
    plt.xticks(word, word, rotation = 'vertical')
    plt.xlabel('Ngrams')
    plt.ylabel('Frequency')
    plt.title('Ngram with Frequency')
    plt.show()
plot_ngrams(bi)

In [ ]:
plot_ngrams(ti)

# **Data Preprocessing**

In [ ]:
#Stopwords
import nltk
nltk.download('stopwords')
stop = set(stopwords.words('english'))

In [ ]:

def lowercase(te):
    tmp = []
    #Complete 
    for i in te : 
        tmp.append(i.lower())
    return tmp
def remove_symbols(text):
    #commplete
    tmp = []
    for i in text: 
        #i = re.sub(r'\n','',i)
       # i = re.sub(r'""', ' ',i)
        i = re.sub(r'[^\w]',' ',i) #Remove all types of symbols from string
        tmp.append(i.replace(') ',''))
    return tmp
def remove_stopwords(text): 
    tmp = []
    for i in text: 
        te = str(i)
        words = word_tokenize(te)
        token_without = [word for word in words if not word in stop]
        s = ''
        for w in token_without : 
            s = s + w + ' '
        tmp.append(s)
    return tmp
def stemming(text): 
    tmp = []
    for i in text : 
        te = str(i)
        lemmatizer = WordNetLemmatizer()
        words = word_tokenize(te)
        s = ''
        for w in words:
            rw = lemmatizer.lemmatize(w)
            s = s + rw + ' '
        tmp.append(s)
    return tmp
def stemming_lem_stop(text):
    tmp = []
    for i in text:
        te = str(i)
        words = word_tokenize(te)
        ps = PorterStemmer()
        lemmatizer = WordNetLemmatizer()
        token_without = [word for word in words if not word in stop] #Remove Stopwords from token
        s = ''
        for w in token_without: 
            rootword = ps.stem(w)
            rw = lemmatizer.lemmatize(rootword) #Lemmatization
            s = s + rw + ' '
        tmp.append(s)
    return tmp



In [ ]:
train1 = lowercase(train_data['comment_text'])
train2 = remove_symbols(train1)
train4 = stemming(train2)
train3 = remove_stopwords(train4)
test1 = lowercase(text)
test2 = remove_symbols(test1)
test4 = stemming(test2)
test3 = remove_stopwords(test4)

# **Calculating Score for Training dataset**
Here, I had calculated the score for training set with the help of TextBlob Sentiment Analysis 
from Jigsaw Toxic Comment Classification Competition Datasets

In [ ]:

pol = []
for i in train3:
    analysis = TextBlob(i)
    x = analysis.sentiment.polarity
    NewValue = (((x - (-1)) * 1) / 2) + 0 #NewValue = (((OldValue - OldMin) * NewRange) / OldRange) + NewMin
    if NewValue == 0.5 :
        pol.append(0)
    else : 
        pol.append(round(NewValue,2))

In [ ]:
len(train3)

In [ ]:
com = []
di = {}
pol1 = []
for i in range(len(data['comment_id'])): 
    com.append(data['comment_id'][i])


# **Data Vectorization**

In [ ]:

tfid = TfidfVectorizer(max_features = 50, min_df= 3, max_df=0.5, analyzer = 'word')
res = tfid.fit_transform(train3).toarray()
res1 = tfid.fit_transform(test3).toarray()
#valid_res = tfid.fit_transform(valid_x).toarray()
# dic = {}
# for e1, e2 in zip(tfid.get_feature_names(), tfid.idf_): 
#     dic[e1] = e2


In [ ]:
res1[0].shape

In [ ]:
# Define initial best params and MAE\
from numpy import arange

params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    #'objective':'reg:linear',
}
gridsearch_params = [
    (max_depth, min_child_weight, eta)
    for max_depth in range(9,12)
    for min_child_weight in range(5,8)
    for eta in arange(0.1,1,0.1)
]
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
dtrain = xgb.DMatrix(res,pol)
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight, eta in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}, eta={}".format(
                             max_depth,
                             min_child_weight,
                             eta ))    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    params['eta'] = eta
 # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        #num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=5
    )    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight, eta)
        print("Best params: {}, {}, {}, MAE: {}".format(best_params[0], best_params[1],best_params[2], min_mae))

In [ ]:
d = best_params[0]
c = best_params[1]
e = best_params[2]

# **Data Modelling**

In [ ]:

model = XGBRegressor(n_estimators = 1000,max_depth = d,min_child_weight= c,eta = e, subsample=0.7, colsample_bytree=0.8)
model.fit(res,pol)
y_pred = abs(model.predict(res1))

In [ ]:
di = {'comment_id': com, 'score': y_pred}
df = pd.DataFrame(di)

In [ ]:
df.head()

In [ ]:
#Output file
df.to_csv("submission.csv", index=False)

**If you like this Notebook, Please upvote.**